## Stuff to load everything

In [1]:
import rootutils

rootutils.setup_root("/TopoBenchmarkX", indicator=".project-root", pythonpath=True)

import hydra
from hydra.experimental import initialize, compose

initialize(config_path="../configs", job_name="job")
config = compose(config_name="train.yaml", return_hydra_config=True)

/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
/usr/local/lib/python3.11/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)


In [4]:
import torch_geometric
import torch

In [5]:
from torch_geometric.utils import k_hop_subgraph
from torch_geometric.utils import to_undirected

In [8]:
data_mutag = torch_geometric.datasets.TUDataset(root="../data/graph", name="MUTAG")
n_graphs = len(data_mutag)
loader = torch_geometric.loader.DataLoader(data_mutag, batch_size=n_graphs)

In [9]:
import copy
class KHopLifting(torch_geometric.transforms.BaseTransform):
    def __init__(self, k=1):
        super().__init__()
        self.k = k

    def forward(self, batch: torch_geometric.data.Batch, slices: dict = None) -> dict:
        data_lifted = copy.copy(batch)

        n_nodes = batch.x.shape[0]

        incidence_1 = torch.zeros(n_nodes, n_nodes)
        edge_index = torch_geometric.utils.to_undirected(batch.edge_index)
        for n in range(n_nodes):
            neighbors, _, _, _ = torch_geometric.utils.k_hop_subgraph(
                n, self.k, edge_index
            )
            incidence_1[n, neighbors] = 1
        incidence_1 = torch.Tensor(incidence_1).to_sparse_coo()
        data_lifted["hyperedges"] = incidence_1
        return data_lifted

    def __call__(self, data, slices=None):
        return self.forward(data, slices)
    
class KNearestNeighborsLifting(torch_geometric.transforms.BaseTransform):
    def __init__(self, k=1):
        super().__init__()
        self.transform = torch_geometric.transforms.KNNGraph(k)

    def forward(self, batch: torch_geometric.data.Batch, slices: dict = None) -> dict:
        batch_lifted = copy.copy(batch)
        batch_lifted.pos = batch_lifted.x
        n_nodes = batch.x.shape[0]
        incidence_1 = torch.zeros(n_nodes, n_nodes)
        print("ok")
        print(batch_lifted.batch)
        data_temp = self.transform(batch_lifted)
        incidence_1[data_temp.edge_index[0],data_temp.edge_index[1]] = 1
        incidence_1 = torch.Tensor(incidence_1).to_sparse_coo()
        batch_lifted["hyperedges"] = incidence_1
        return batch_lifted

    def __call__(self, data, slices=None):
        return self.forward(data, slices)

In [11]:
lift = KHopLifting(k=1)

for d in loader:
    print(d)
    data_lifted = lift(d)

print(data_mutag)
print(data_lifted)

DataBatch(edge_index=[2, 7442], x=[3371, 7], edge_attr=[7442, 4], y=[188], batch=[3371], ptr=[189])
MUTAG(188)
DataBatch(edge_index=[2, 7442], x=[3371, 7], edge_attr=[7442, 4], y=[188], batch=[3371], ptr=[189], hyperedges=[3371, 3371])


In [12]:
print(data_lifted.hyperedges.indices()[1][data_lifted.hyperedges.indices()[0] == 0])

tensor([0, 1, 5])


In [13]:
def get_cora(cfg):
    data_dir = cfg["data_dir"]
    print(f"Loading {cfg['data_domain']} dataset name: {cfg['data_name']}")

    data = torch_geometric.datasets.Planetoid(data_dir, "Cora")
    return data

In [14]:
from torch_geometric.loader.dataloader import DataLoader

data_cora = get_cora(config.data)
n_graphs = len(data_cora)
loader = torch_geometric.loader.DataLoader(data_cora, batch_size=n_graphs)

Loading hypergraph dataset name: cora


In [16]:
for batch in loader:
    data_lifted = lift(batch)

print(data_cora)
print(data_lifted)

Cora()
DataBatch(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], batch=[2708], ptr=[2], hyperedges=[2708, 2708])


In [17]:
print(data_lifted.hyperedges.indices()[1][data_lifted.hyperedges.indices()[0] == 2])

tensor([   1,    2,  332, 1454, 1666, 1986])


In [18]:
model = hydra.utils.instantiate(config.model)

/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'readout_workaround' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['readout_workaround'])`.
/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'readout' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['readout'])`.


In [19]:
results = model(data_lifted.x, data_lifted.hyperedges)

In [20]:
print(results)

(tensor([[1.1128, 0.0000, 0.0000,  ..., 0.0000, 1.0280, 0.0000],
        [1.1101, 0.0000, 0.0000,  ..., 0.0000, 0.8908, 0.0000],
        [1.2814, 0.0000, 0.0000,  ..., 0.0000, 1.4679, 0.0000],
        ...,
        [0.8512, 0.0000, 0.0000,  ..., 0.0000, 1.5145, 0.0000],
        [1.0093, 0.0000, 0.0000,  ..., 0.0000, 1.0170, 0.0000],
        [1.0837, 0.0000, 0.0000,  ..., 0.0000, 0.9484, 0.1635]],
       grad_fn=<MulBackward0>), tensor([[0.0000, 0.0000, 0.8117,  ..., 0.0000, 1.9955, 0.0000],
        [0.0000, 0.0000, 0.6481,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 2.2122, 0.0000],
        ...,
        [0.0000, 0.0000, 0.6992,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.8787,  ..., 0.0000, 1.6070, 0.0000],
        [0.0000, 0.0000, 0.8623,  ..., 0.0000, 2.3690, 0.0000]],
       grad_fn=<MulBackward0>))
